In [68]:
import numpy as np
import random
import nbimporter
import _TM as TM
import Encoder
import _SP3 as SP3

In [69]:
encoder = Encoder.ScalarEncoder(out_size=100, w=21, max_val=30)
sp = SP3.SpatialPooler(encoder.out_size, columns=512, perm_conn=.6, minOver=encoder.w/5, potential_rate=0.9)

In [70]:
for epoch in range(3):
    a = np.arange(encoder.max_val)
    random.shuffle(a)
#     print(a)
    for i in a:
        input_data = encoder.encode(i)        
        sp.compute_SDR(input_data)

In [84]:
for i in range(encoder.max_val):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sdr1 = sp.getActiveColumns()
    sp.compute_SDR(encoder.encode(i+1), learn=False)
    sdr2 = sp.getActiveColumns()
    
    sp.similarity(sdr1, sdr2, False)
    
print("columns{}".format(sp.getActiveColumns()))

유사도 : 0.5238095238095238
[  5  14  17  47  53 137 152 154 167 241 277 306 313 326 355 364 370 423
 446 505 507]
[ 47  53 154 167 241 313 355 370 423 446 505]
유사도 : 0.6363636363636364
[ 47  53 154 167 241 313 355 370 423 446 505]
[ 47  64 126 154 180 228 239 241 355 370 374 396 401 402 433 446 453 505]
유사도 : 0.5555555555555556
[ 47  64 126 154 180 228 239 241 355 370 374 396 401 402 433 446 453 505]
[ 47 152 154 180 241 244 306 353 355 370 374 405 422 446 453 505]
유사도 : 0.6875
[ 47 152 154 180 241 244 306 353 355 370 374 405 422 446 453 505]
[ 47 152 154 180 241 293 306 355 370 374 422 446 489 495]
유사도 : 0.35714285714285715
[ 47 152 154 180 241 293 306 355 370 374 422 446 489 495]
[ 44  47  49 106 120 126 147 154 244 255 306 349 374 495 499 505]
유사도 : 0.625
[ 44  47  49 106 120 126 147 154 244 255 306 349 374 495 499 505]
[ 47  49  53  60  77 120 126 154 196 255 301 313 319 349 374 495 499]
유사도 : 0.5882352941176471
[ 47  49  53  60  77 120 126 154 196 255 301 313 319 349 374 495 499]
[ 

In [87]:
tm = TM.TemporalMemory(sdr_size=sp.columnCount, max_seg=5, threshold=7)

tm.learn = True

seq_data = [1,2,3,4,5]

for i in range(10):
    for s in seq_data:
        dat = encoder.encode(s)
        sp.compute_SDR(dat, learn=False)
        sdr = sp.getActiveColumns_raw()        
        tm.compute(sdr)

In [88]:
tm.learn = False
prev_pred = []
for d in seq_data:
    dat = encoder.encode(d)
    sp.compute_SDR(dat, learn=False)
    sdr = sp.getActiveColumns_raw()
    tm.compute(sdr)
    tm.test(prev_pred, sp.getActiveColumns())
    prev_pred = tm.column_predicted()
    
    print("Now columns (sdr) : {}".format(sp.getActiveColumns()))
    print("Predicted columns (예측Predict) : {}".format(tm.column_predicted()))
    print("Now burst : {}".format(tm.column_bursted()))
    print("\n")
    
    

예측 Accuracy : 0.0
Now columns (sdr) : [ 68 151 167 215 279 355 423 446 475 505]
Predicted columns (예측Predict) : [355, 68, 423, 492, 80, 242, 370, 446]
Now burst : [475, 167]


예측 Accuracy : 0.7272727272727273
Now columns (sdr) : [ 34  68  80 242 355 370 423 446 475 492 505]
Predicted columns (예측Predict) : [34, 355, 344, 327, 76, 492, 80, 370, 83, 152, 475, 446]
Now burst : [505, 34, 475]


예측 Accuracy : 1.0
Now columns (sdr) : [ 34  76  80  83 152 327 344 370 446 475]
Predicted columns (예측Predict) : [34, 6, 105, 489, 76, 83, 344, 125]
Now burst : [34, 83]


예측 Accuracy : 0.8
Now columns (sdr) : [  6  34  39  76  83 105 125 344 489 495]
Predicted columns (예측Predict) : [356, 105, 489, 112, 154, 125]
Now burst : [344, 495, 125, 39]


예측 Accuracy : 0.6
Now columns (sdr) : [  6  39  76 105 112 125 154 356 489 510]
Predicted columns (예측Predict) : [355, 68, 423, 215, 151, 505, 279, 446]
Now burst : [76, 510, 6, 39]




In [129]:
class HTM:
    encoder #encoder for the HTM
    sp  #spatial Pooler for the HTM
    tm  # Temporal or 'Sequence' Memory for the HTM
    prev_pred = [] # previous prediction from the temporal memory as of the last step
    sp_learn = False
    tm_learn = True
    
    def __init__(self): #TODO set what values need to be initialized here
        self.encoder = Encoder.ScalarEncoder(out_size=100, w=21, max_val=30) #set what values the encoder is to take here
        self.sp = SP3.SpatialPooler(encoder.out_size, columns=512, perm_conn=.6, minOver=encoder.w/5, potential_rate=0.9)
        
        self.tm = TM.TemporalMemory(sdr_size=self.sp.columnCount, max_seg=5, threshold=7)
           
        
    def compute(self, data):
        self.tm.learn = False
        dat = encoder.encode(data)
        self.sp.compute_SDR(dat, learn = self.sp_learn)
        sdr = sp.getActiveColumns_raw()
        self.tm.compute(sdr)
        self.prev_pred = tm.column_predicted()
        
    def prediction(self, printable = False):              
        if(printable == True):
            self.tm.test(self.prev_pred, self.sp.getActiveColumns())
            print("현재 columns (sdr) : {}".format(sp.getActiveColumns()))
            print("다음 columns (예측) : {}".format(tm.column_predicted()))
            print("현재 burst : {}".format(tm.column_bursted()))
            print("\n")
            
        return (len(np.intersect1d(self.prev_pred, self.sp.getActiveColumns())) / len(self.sp.getActiveColumns()))
        
        
        
# prediction error calculation
prediction_error =None
window = 10 # window size to comapare the prediction error against.
        
def prediction_error():
        
    return 0



In [132]:
htm = HTM()

for j in range (10):
    for i in range (10):
        htm.compute(data = i)
        print("number:{}".format(i))
        print(htm.prediction(printable=False))
    print("epoch {}".format(j))


number:0
0.0
number:1
0.0
number:2
0.0
number:3
0.0
number:4
0.0
number:5
0.0
number:6
0.0
number:7
0.0
number:8
0.0
number:9
0.043478260869565216
epoch 0
number:0
0.0
number:1
0.0
number:2
0.0
number:3
0.0
number:4
0.0
number:5
0.0
number:6
0.0
number:7
0.0
number:8
0.0
number:9
0.043478260869565216
epoch 1
number:0
0.0
number:1
0.0
number:2
0.0
number:3
0.0
number:4
0.0
number:5
0.0
number:6
0.0
number:7
0.0
number:8
0.0
number:9
0.043478260869565216
epoch 2
number:0
0.0
number:1
0.0
number:2
0.0
number:3
0.0
number:4
0.0
number:5
0.0
number:6
0.0
number:7
0.0
number:8
0.0
number:9
0.043478260869565216
epoch 3
number:0
0.0
number:1
0.0
number:2
0.0
number:3
0.0
number:4
0.0
number:5
0.0
number:6
0.0
number:7
0.0
number:8
0.0
number:9
0.043478260869565216
epoch 4
number:0
0.0
number:1
0.0
number:2
0.0
number:3
0.0
number:4
0.0
number:5
0.0
number:6
0.0
number:7
0.0
number:8
0.0
number:9
0.043478260869565216
epoch 5
number:0
0.0
number:1
0.0
number:2
0.0
number:3
0.0
number:4
0.0
numbe